In [2]:
# ------------------------------------------------------------------------------ #
# @Author:        L. Rudelt
# @Email:         lucas.rudelt@ds.mpg.de
# @Created:       2023-11-16 16:49:35
# @Last Modified: 
# ------------------------------------------------------------------------------ #



import logging
logging.basicConfig(
    format="%(asctime)s | %(levelname)-8s | %(name)-s | %(funcName)-s | %(message)s",
    level=logging.WARNING,
)
log = logging.getLogger("notebook")
log.setLevel("DEBUG")

import re
import glob
import h5py
import sys
import os
import numpy as np
import xarray as xr
import pandas as pd


# also needs to be added for each dask-worker
extra_path = os.path.abspath('../')
sys.path.append(extra_path)
log.info(f"project directory: {extra_path}")

from ana import utility as utl
from ana import plot_helper as ph

ph.log.setLevel("DEBUG")
utl.log.setLevel("DEBUG")

data_dir = "../data/"

2023-11-17 12:05:33,849 | INFO     | notebook | <module> | project directory: /data.nst/lucas/projects/mouse_visual_timescales_predictability/paper_code/experiment_analysis


In [7]:
# Load ccf coordinates
from pathlib import Path
from allensdk.core.reference_space_cache import ReferenceSpaceCache
from allensdk.brain_observatory.ecephys.ecephys_project_cache import EcephysProjectCache

import re

def get_layer_name(acronym):
    try:
        layer = int(re.findall(r'\d+', acronym)[0])
        if layer == 3:
            layer = 0
        return layer
    except IndexError:
        return 0

atlas_dir = '/project.nst/neuroscience-raw/Allen/allen_brain_atlas'
data_directory = '/project.nst/neuroscience-raw/Allen/visual_coding_neuropixels'

manifest_path = os.path.join(data_directory, "manifest.json")

cache = EcephysProjectCache.from_warehouse(manifest=manifest_path)

reference_space_key = os.path.join('annotation', 'ccf_2017')
resolution = 10

rspc = ReferenceSpaceCache(resolution, reference_space_key, manifest=Path(atlas_dir) / 'manifest.json')
# ID 1 is the adult mouse structure graph
tree = rspc.get_structure_tree(structure_graph_id=1) 

# get unit information 
# TODO: adapt default filters to include all the units that we included in the analysis
units = cache.get_units()

# get annotation volume 
annotation, meta = rspc.get_annotation_volume()




In [26]:
# Compute layer information for each unit
layers = []
units_with_layer = []
structure_acronyms = [] 
for unit in units.index:
    unit_data = units.loc[unit]
    # get structure id from ccf coordinates and annotation volume 
    x = np.amax((unit_data.anterior_posterior_ccf_coordinate / 10).astype('int'), 0)
    y = np.amax((unit_data.dorsal_ventral_ccf_coordinate / 10).astype('int'), 0)
    z = np.amax((unit_data.left_right_ccf_coordinate / 10).astype('int'), 0)
    try:
        structure_id = annotation[x, y, z]
        structure = tree.get_structures_by_id([structure_id])
        structure_acronym = structure[0]["acronym"]
        structure_acronyms += [structure_acronym]
        layers += [get_layer_name(structure_acronym)]
        units_with_layer += [unit] 
    except:
        print(x,y,z)



-9223372036854775808 -9223372036854775808 -9223372036854775808
-9223372036854775808 -9223372036854775808 -9223372036854775808
-9223372036854775808 -9223372036854775808 -9223372036854775808
-9223372036854775808 -9223372036854775808 -9223372036854775808
-9223372036854775808 -9223372036854775808 -9223372036854775808
-9223372036854775808 -9223372036854775808 -9223372036854775808
-9223372036854775808 -9223372036854775808 -9223372036854775808
-9223372036854775808 -9223372036854775808 -9223372036854775808
-9223372036854775808 -9223372036854775808 -9223372036854775808
-9223372036854775808 -9223372036854775808 -9223372036854775808
-9223372036854775808 -9223372036854775808 -9223372036854775808
-9223372036854775808 -9223372036854775808 -9223372036854775808
-9223372036854775808 -9223372036854775808 -9223372036854775808
-9223372036854775808 -9223372036854775808 -9223372036854775808
-9223372036854775808 -9223372036854775808 -9223372036854775808
-9223372036854775808 -9223372036854775808 -922337203685

In [ ]:
# Check how many units are found for each layer
layers = np.array(layers)
for layer in np.unique(layers):
    print(layer, len(layers[layers == layer]))

In [40]:
# Create data_frame with units_in_both as index and layers as column, add values from array layers
df = pd.DataFrame(index=units_with_layer, columns=["layer"])
df = df.assign(layer=layers)
# save to disk as csv file
df.to_csv("../data/layers.csv")

In [28]:
for acronym in np.unique(structure_acronyms):
    print(acronym, get_layer_name(acronym))

APN 0
AUDd2/3 2
AUDd4 4
AUDd5 5
AUDd6a 6
AUDd6b 6
AUDp2/3 2
AUDp4 4
AUDp5 5
AUDp6a 6
AUDp6b 6
AUDpo1 1
AUDpo2/3 2
AUDpo4 4
AUDpo5 5
AUDpo6a 6
AUDpo6b 6
CA1 1
CA2 2
CA3 0
DG-mo 0
DG-po 0
DG-sg 0
Eth 0
HPF 0
IGL 0
IntG 0
LD 0
LGd-co 0
LGd-ip 0
LGd-sh 0
LGv 0
LP 0
LT 0
MB 0
MGd 0
MGm 0
MGv 0
MRN 0
NOT 0
OP 0
PIL 0
PO 0
POL 0
POST 0
PP 0
PPT 0
PRE 0
PoT 0
ProS 0
RSPagl2/3 2
RSPagl5 5
RSPagl6a 6
RSPd6a 6
RSPv6a 6
RT 0
SCig 0
SCiw 0
SCop 0
SCsg 0
SGN 0
SSp-bfd1 1
SSp-bfd2/3 2
SSp-bfd4 4
SSp-bfd5 5
SSp-bfd6a 6
SSp-bfd6b 6
SSp-tr2/3 2
SSp-tr4 4
SSp-tr5 5
SSp-tr6a 6
SSp-tr6b 6
SSs2/3 2
SSs4 4
SSs5 5
SSs6a 6
SSs6b 6
SUB 0
TEa1 1
TEa2/3 2
TEa4 4
TEa5 5
TEa6a 6
TEa6b 6
TH 0
VISa1 1
VISa2/3 2
VISa4 4
VISa5 5
VISa6a 6
VISa6b 6
VISal1 1
VISal2/3 2
VISal4 4
VISal5 5
VISal6a 6
VISal6b 6
VISam1 1
VISam2/3 2
VISam4 4
VISam5 5
VISam6a 6
VISam6b 6
VISl2/3 2
VISl4 4
VISl5 5
VISl6a 6
VISl6b 6
VISli2/3 2
VISli4 4
VISli5 5
VISli6a 6
VISli6b 6
VISp1 1
VISp2/3 2
VISp4 4
VISp5 5
VISp6a 6
VISp6b 6
VISpm2/3 2
VISpm

In [36]:
df

,layer
950907205,NaN
950907209,NaN
950907524,NaN
950907216,NaN
950907220,NaN
...,...
951911942,NaN
951911948,NaN
951911955,NaN
951911961,NaN
